In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array
from PIL import Image


In [ ]:
SIZE = 128 
NoofIMGs = 9001
batch_size = 16

path_1 = '/kaggle/input/face-mask-lite-dataset/without_mask'
path_2 = '/kaggle/input/face-mask-lite-dataset/with_mask'


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

files_1 = sorted_alphanumeric([f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1, f))])[:NoofIMGs]
files_2 = sorted_alphanumeric([f for f in os.listdir(path_2) if os.path.isfile(os.path.join(path_2, f))])[:NoofIMGs]
files_3 = sorted_alphanumeric([f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1, f))])[NoofIMGs:]
files_4 = sorted_alphanumeric([f for f in os.listdir(path_2) if os.path.isfile(os.path.join(path_2, f))])[NoofIMGs:]
without_mask_paths = [os.path.join(path_1, fname) for fname in files_1]
with_mask_paths = [os.path.join(path_2, fname) for fname in files_2]
without_mask_val_paths=[os.path.join(path_1, fname) for fname in files_3]
with_mask_val_paths = [os.path.join(path_2, fname) for fname in files_4]

def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')  
    img = img.resize((SIZE, SIZE))  
    img = np.array(img) / 127.5 - 1.0  
    return tf.convert_to_tensor(img, dtype=tf.float32)


def create_dataset(without_mask_paths, with_mask_paths):
    def generator():
        for without_mask_path, with_mask_path in zip(without_mask_paths, with_mask_paths):
            without_mask_img = load_and_preprocess_image(without_mask_path)
            with_mask_img = load_and_preprocess_image(with_mask_path)
            yield without_mask_img, with_mask_img

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(SIZE, SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(SIZE, SIZE, 3), dtype=tf.float32),
        )
    )
    return dataset

train_dataset = create_dataset(without_mask_paths, with_mask_paths)
val_dataset=create_dataset(without_mask_val_paths, with_mask_val_paths)


train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

for without_mask_img, with_mask_img in train_dataset.take(1):
    print("Without mask image shape:", without_mask_img.shape)
    print("With mask image shape:", with_mask_img.shape)
for without_mask_img, with_mask_img in val_dataset.take(1):
    print("Without mask image shape:", without_mask_img.shape)
    print("With mask image shape:", with_mask_img.shape)


In [ ]:
for i in range(4):
    plt.subplot(2, 5, i + 1)
    plt.imshow((without_mask_img[i].numpy() + 1) / 2)  # Chuyển ảnh về [0, 1]
    plt.title("UnMasked")
    plt.axis('off')

    plt.subplot(2, 5, i + 6)
    plt.imshow((with_mask_img[i].numpy() + 1) / 2)  # Chuyển ảnh về [0, 1]
    plt.title("Mask")
    plt.axis('off')

plt.show()


In [ ]:
# Double Convolutional Block
def double_conv(in_channels, out_channels):
    return tf.keras.Sequential([
        layers.Conv2D(out_channels, kernel_size=3, padding='same', activation=None),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(out_channels, kernel_size=3, padding='same', activation=None),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

def Generator(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    conv_1 = double_conv(3, 64)(inputs)
    pool_1 = layers.MaxPooling2D(pool_size=(2, 2))(conv_1)

    conv_2 = double_conv(64, 128)(pool_1)
    pool_2 = layers.MaxPooling2D(pool_size=(2, 2))(conv_2)

    conv_3 = double_conv(128, 256)(pool_2)
    pool_3 = layers.MaxPooling2D(pool_size=(2, 2))(conv_3)

    conv_4 = double_conv(256, 512)(pool_3)
    pool_4 = layers.MaxPooling2D(pool_size=(2, 2))(conv_4)

    conv_5 = double_conv(512, 1024)(pool_4)

    # Decoder
    upconv_1 = layers.Conv2DTranspose(512, kernel_size=2, strides=2, padding='same')(conv_5)
    concat_1 = layers.Concatenate()([upconv_1, conv_4])
    conv_6 = double_conv(1024, 512)(concat_1)

    upconv_2 = layers.Conv2DTranspose(256, kernel_size=2, strides=2, padding='same')(conv_6)
    concat_2 = layers.Concatenate()([upconv_2, conv_3])
    conv_7 = double_conv(512, 256)(concat_2)

    upconv_3 = layers.Conv2DTranspose(128, kernel_size=2, strides=2, padding='same')(conv_7)
    concat_3 = layers.Concatenate()([upconv_3, conv_2])
    conv_8 = double_conv(256, 128)(concat_3)

    upconv_4 = layers.Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(conv_8)
    concat_4 = layers.Concatenate()([upconv_4, conv_1])
    conv_9 = double_conv(128, 64)(concat_4)

    output = layers.Conv2D(3, kernel_size=3, strides=1, padding='same', activation='tanh')(conv_9)

    return tf.keras.Model(inputs, output)

generator = tf.keras.models.load_model('/kaggle/input/asdfasdf/tensorflow2/default/1/generator_final.h5')

# Test generator 
dummy_input = tf.random.normal([1, 128, 128, 3])  # Batch of 1, 128x128 RGB image
output = generator(dummy_input)
print("Generator output shape:", output.shape)




In [ ]:
for without_mask_imgs, with_mask_imgs in train_dataset.take(1):
    # Test generator trên ảnh không có khẩu trang
    generated_imgs = generator(with_mask_imgs)

    print(f"Input batch shape: {without_mask_imgs.shape}")
    print(f"Generated batch shape: {generated_imgs.shape}")


In [ ]:

def denormalize(img):
    return (img + 1) / 2 

plt.figure(figsize=(8, 4))

# Hiển thị ảnh đầu vào không có khẩu trang
plt.subplot(1, 2, 1)
plt.imshow(denormalize(without_mask_imgs[0].numpy()))  # Ảnh đầu tiên trong batch
plt.title("Without Mask (Input)")
plt.axis('off')

# Hiển thị ảnh được tạo ra bởi generator
plt.subplot(1, 2, 2)
plt.imshow(denormalize(generated_imgs[0].numpy()))  # Ảnh đầu tiên trong batch
plt.title("Generated Image")
plt.axis('off')

plt.show()


In [ ]:
from tensorflow.keras import layers

def Discriminator(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=4, strides=2, padding="same", kernel_initializer="he_normal")(inputs)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding="same", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding="same", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(512, kernel_size=4, strides=2, padding="same", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)


    x = layers.Flatten()(x)
    x = layers.Dense(1, activation="sigmoid")(x)  # Output là xác suất thật/giả

    return tf.keras.Model(inputs, x)

# Instantiate the discriminator
discriminator = tf.keras.models.load_model('/kaggle/input/asdfasdf/tensorflow2/default/1/discriminator.h5')
discriminator.summary()

In [10]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = False)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4)


In [11]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)
def discriminator_loss(fake_output, real_output):
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    return fake_loss + real_loss

In [12]:
@tf.function
def train_step(without_mask_imgs, with_mask_imgs):
    
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        
        generated_imgs = generator(with_mask_imgs, training=True)

        
        real_output = discriminator(without_mask_imgs, training=True)
        fake_output = discriminator(generated_imgs, training=True)

        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(fake_output, real_output)

    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [13]:
@tf.function
def val_step(without_mask_imgs, with_mask_imgs):
    
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        
        generated_imgs = generator(with_mask_imgs, training=False)

        
        real_output = discriminator(without_mask_imgs, training=False)
        fake_output = discriminator(generated_imgs, training=False)

        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(fake_output, real_output)

    return gen_loss, disc_loss

In [ ]:
import time


EPOCHS = 20
for epoch in range(EPOCHS):
    start = time.time()
    print("\nEpoch : {}".format(epoch + 1))


    total_gen_loss_train = 0.0
    total_dis_loss_train = 0.0
    total_gen_loss_val = 0.0
    total_dis_loss_val = 0.0
    num_batches_train= 0
    num_batches_val=0

    for without_mask_imgs, with_mask_imgs in train_dataset:
        gen_loss, disc_loss = train_step(without_mask_imgs, with_mask_imgs)
        total_gen_loss_train += gen_loss
        total_dis_loss_train += disc_loss
        num_batches_train += 1
    
    
    avg_gen_loss_train = total_gen_loss_train / num_batches_train
    avg_dis_loss_train = total_dis_loss_train / num_batches_train


    
    for without_mask_imgs, with_mask_imgs in val_dataset:
        gen_loss, disc_loss = val_step(without_mask_imgs, with_mask_imgs)
        total_gen_loss_val += gen_loss
        total_dis_loss_val += disc_loss
        num_batches_val += 1
    avg_gen_loss_val = total_gen_loss_val / num_batches_val
    avg_dis_loss_val = total_dis_loss_val / num_batches_val

    
    if (epoch + 1) % 5 == 0:
        for without_mask_imgs, _ in train_dataset.take(1):
            generated_imgs = generator(with_mask_imgs)
            plt.imshow((generated_imgs[0].numpy() + 1) / 2)  # Hiển thị ảnh đầu tiên trong batch
            plt.axis('off')
            plt.show()

    print(" Time:{}".format(np.round(time.time() - start, 2)))
    print("Avg Generator Loss Train: {:.4f} Avg Discriminator Loss Train: {:.4f}  Avg Generator Loss Val: {:.4f} Avg Discriminator Loss Val: {:.4f}".format(avg_gen_loss_train, avg_dis_loss_train,avg_gen_loss_val,avg_dis_loss_val))


In [ ]:
generator.save('/kaggle/working/generator_new_final.h5')
